In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product
os.chdir("../")
from src.models.pipelines import Pipeline

In [ ]:
pd.set_option('display.max_columns', None)
sns.set_theme()

# Datasets Description

# Prediction Global Report

In [ ]:
_list_datasets_T = [["mrda", 50]]

_list_encoder = ["bert-base-cased", "xlnet-base-cased", "prajjwal1/bert-small"]

_list_data_format_decoder = [["MLP", "concatenate"], ["GRU", "separate"]]

_list_inputs = list(product(_list_datasets_T, _list_encoder, _list_data_format_decoder))

_possible_pipelines = {
                            "inputs": _list_inputs, 
                            "outputs": []
                        }

df_global_report = pd.DataFrame()

In [ ]:
_possible_pipelines

In [ ]:
for _input in _possible_pipelines["inputs"]:
    _pipeline = Pipeline(_input[0][0], _input[0][1], _input[1], _input[2][0], _input[2][1])
    _pipeline.summarize()
    df_global_report = pd.concat([df_global_report, _pipeline.df_report]).reset_index(drop=True)
    _possible_pipelines["outputs"].append(_pipeline.confusion_matrix)

In [ ]:
df_global_report

In [ ]:
_possible_pipelines["outputs"]